In [1]:
import undetected_chromedriver as uc
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

Fonction Choisir lieu du drive

In [2]:
def select_city(driver,city):
    driver.get('https://www.auchan.fr/')
    driver.maximize_window()
    try:
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'onetrust-accept-btn-handler'))
        )
        button.click()
        time.sleep(3)
        
        location = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'journey-reminder-header'))
        )
        location.click()
        time.sleep(2)
        

        search_location = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, 'query'))
        )
        search_location.click()
        time.sleep(2)

        search_location.send_keys(city)
        time.sleep(4)
        
        search_location.send_keys(Keys.ARROW_DOWN, Keys.RETURN)
        time.sleep(3)

        bt_choisir_drive = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="journey-update-modal_desc"]/div[1]/div[2]/div[2]/section/div[1]/div/div/div[2]/form/button'))
        )
        bt_choisir_drive.click()
        time.sleep(2)
        

        bt_rayon = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="navigation"]'))
        )
        bt_rayon.click()
        time.sleep(2)
    except:
        driver.quit()

Fonction search rayons principaux

In [3]:
def search_rayons_p(driver,list_rayons_principal=[]):
    rayon_tag = driver.find_elements(by = By.CLASS_NAME, value= 'navigation-layer__link')

    for rayon in rayon_tag:

        #search nom rayon principal
        rayon_principal = rayon.text
             
            
        lien_rayon_p = rayon.get_attribute('href')

        time.sleep(2)         

        #ajout nom et lien des rayons principales dans une liste
        dict_rayon_principal={
        'rayon principal' : rayon_principal,
        'lien rayon principal' : lien_rayon_p
            
        }
        list_rayons_principal.append(dict_rayon_principal)
    df_rayonsp = pd.DataFrame(list_rayons_principal)
    return df_rayonsp

Fonction search rayons principal et secondaire

In [4]:

def search_rayon_sec(driver,df_rayonsp,list_rayons=[]):
    for i, lien_rayon in enumerate(df_rayonsp['lien rayon principal']):
        driver.get(lien_rayon)
        time.sleep(5)
        


        rayons_sec_tag = driver.find_elements(by = By.CLASS_NAME, value= 'universe-block')
        if len(rayons_sec_tag) == 0:
            dict_rayons={
                                'rayon principal' : df_rayonsp.iloc[i]['rayon principal'],
                                'lien rayon principal' : df_rayonsp.iloc[i]['lien rayon principal'],
                                'rayon secondaire': df_rayonsp.iloc[i]['rayon principal'],
                                'lien rayon secondaire' : df_rayonsp.iloc[i]['lien rayon principal']
                            }
            print(dict_rayons)
            list_rayons.append(dict_rayons)
        else:           
            for rayon_sec in rayons_sec_tag :
                nom_rayon_sec = rayon_sec.find_element(by = By.CLASS_NAME, value= 'universe-block__title').text
                lien_rayon_sec = rayon_sec.find_element(by = By.TAG_NAME, value ='a')

                dict_rayons={
                                'rayon principal' : df_rayonsp.iloc[i]['rayon principal'],
                                'lien rayon principal' : df_rayonsp.iloc[i]['lien rayon principal'],
                                'rayon secondaire': nom_rayon_sec,
                                'lien rayon secondaire' : lien_rayon_sec.get_attribute('href')
                            }
                print(dict_rayons)
                list_rayons.append(dict_rayons)
                    
    df_rayons = pd.DataFrame(list_rayons)
    return(df_rayons)

        

Search produits on 1 page

In [5]:
def page_produits(produits_tag,df_rayons, i, list_page_produits =[]):
     for produit in produits_tag:

                prix_produit = produit.find_element(by = By.CSS_SELECTOR, value= 'footer > div.product-thumbnail__footer-wrapper > div.product-thumbnail__price.product-price__container > div')
                nom_produit = produit.find_element(by = By.CLASS_NAME, value= 'product-thumbnail__description')  
                attribut_produit = produit.find_element(by = By.CLASS_NAME, value= 'product-thumbnail__attributes')
                    
                dict_produit={
                                'rayon_principal' : df_rayons.iloc[i]['rayon principal'],
                                'lien_rayon principal' : df_rayons.iloc[i]['lien rayon principal'],
                                'rayon_secondaire': df_rayons.iloc[i]['rayon secondaire'],
                                'lien_rayon_secondaire' : df_rayons.iloc[i]['lien rayon secondaire'],
                                'nom_produit': nom_produit.text,
                                'prix_produit': prix_produit.text,
                                'attribut_produit' : attribut_produit.text
                                }
    
                print(dict_produit)
                list_page_produits.append(dict_produit)
     return(list_page_produits)

Search all produits 1

In [6]:
def search_produits1(driver,df_rayons,list_produits=[]):
    for i, lien_rayon in enumerate(df_rayons['lien rayon secondaire']):

        if (i<1) :

            print(i)
            print(lien_rayon)
            driver.get(lien_rayon)
            time.sleep(3)


            produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

            list_page_produits = page_produits(produits_tag,df_rayons,i)
            for produit in list_page_produits:
                 list_produits.append(produit)
            


            driver.execute_script("window.scrollTo(0, 10)")

            try : 
                pages = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav/a'))
                )
                pages.click()
                time.sleep(3)
                

                try :
                    for y in range(200) :
                        time.sleep(3)
                        driver.execute_script("window.scrollTo(0, 10)")
                        time.sleep(3)
                        produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

                        list_page_produits = page_produits(produits_tag,df_rayons,i)
                        for produit in list_page_produits:
                            list_produits.append(produit)
                    

                        pages = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav[2]/a[2]'))
                        )
                        pages.click()
                except:
                        pass
            except:
                 pass
         
        else :
            pass

    df_produits = pd.DataFrame(list_produits)
    df_produits.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    df_produits.to_csv('Auchan_produits1.csv')
    return(df_produits)



        

Search all produit 2

In [7]:
def search_produits2(driver,df_rayons,list_produits=[]):
    for i, lien_rayon in enumerate(df_rayons['lien rayon secondaire']):

        if ((i>51)&(i<65)) :

            print(i)
            print(lien_rayon)
            driver.get(lien_rayon)
            time.sleep(3)


            produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

            list_page_produits = page_produits(produits_tag,df_rayons,i)
            list_produits = list_produits + list_page_produits


            driver.execute_script("window.scrollTo(0, 10)")

            try : 
                pages = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav/a'))
                )
                pages.click()
                time.sleep(3)
                

                try :
                    for y in range(200) :
                        time.sleep(3)
                        driver.execute_script("window.scrollTo(0, 10)")
                        time.sleep(3)
                        produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

                        list_page_produits = page_produits(produits_tag,df_rayons,i)
                        list_produits = list_produits + list_page_produits
                    
                         
                        pages = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav[2]/a[2]'))
                        )
                        pages.click()
                except:
                        pass
            except:
                 pass
         
        else :
            pass

    df_produits = pd.DataFrame(list_produits)
    df_produits.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    df_produits.to_csv('Auchan_produits2.csv')
    return(df_produits)


Search produits 3

In [8]:
def search_produits3(driver,df_rayons,list_produits=[]):
    for i, lien_rayon in enumerate(df_rayons['lien rayon secondaire']):

        if ((i>65)&(i<76)) :

            print(i)
            print(lien_rayon)
            driver.get(lien_rayon)
            time.sleep(3)


            produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

            list_page_produits = page_produits(produits_tag,df_rayons,i)
            list_produits = list_produits + list_page_produits


            driver.execute_script("window.scrollTo(0, 10)")

            try : 
                pages = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav/a'))
                )
                pages.click()
                time.sleep(3)
                

                try :
                    for y in range(200) :
                        time.sleep(3)
                        driver.execute_script("window.scrollTo(0, 10)")
                        time.sleep(3)
                        produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

                        list_page_produits = page_produits(produits_tag,df_rayons,i)
                        list_produits = list_produits + list_page_produits
                    
                         
                        pages = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav[2]/a[2]'))
                        )
                        pages.click()
                except:
                        pass
            except:
                 pass
         
        else :
            pass

    df_produits = pd.DataFrame(list_produits)
    df_produits.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    df_produits.to_csv('Auchan_produits3.csv')
    return(df_produits)

Search produits 4

In [9]:
def search_produits4(driver,df_rayons,list_produits=[]):
    for i, lien_rayon in enumerate(df_rayons['lien rayon secondaire']):

        if (i>77) :

            print(i)
            print(lien_rayon)
            driver.get(lien_rayon)
            time.sleep(3)


            produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

            list_page_produits = page_produits(produits_tag,df_rayons,i)
            list_produits = list_produits + list_page_produits


            driver.execute_script("window.scrollTo(0, 10)")

            try : 
                pages = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav/a'))
                )
                pages.click()
                time.sleep(3)
                

                try :
                    for y in range(200) :
                        time.sleep(3)
                        driver.execute_script("window.scrollTo(0, 10)")
                        time.sleep(3)
                        produits_tag = driver.find_elements(by = By.CLASS_NAME, value= 'product-thumbnail__content-wrapper')

                        list_page_produits = page_produits(produits_tag,df_rayons,i)
                        list_produits = list_produits + list_page_produits
                    
                         
                        pages = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, '//*[@id="wrapper"]/nav[2]/a[2]'))
                        )
                        pages.click()
                except:
                        pass
            except:
                 pass
         
        else :
            pass

    df_produits = pd.DataFrame(list_produits)
    df_produits.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    df_produits.to_csv('Auchan_produits4.csv')
    return(df_produits)

In [10]:

if __name__ == '__main__':
    options = Options()
    options.add_experimental_option('detach', True)

    driver = uc.Chrome() 
    city ="82000"
    
    #choisir ville
    select_city(driver,city)

    #scraping rayon principal
    df_rayonsp = search_rayons_p(driver)

    #scrap tous les rayons
    df_rayons = search_rayon_sec(driver,df_rayonsp)
    df_rayons.to_csv('Auchan_rayons.csv')
    
    #scrap tous les produits
    df_produits1 = search_produits1(driver,df_rayons)
    df_produits2 = search_produits2(driver,df_rayons)
    df_produits3 = search_produits3(driver,df_rayons)
    df_produits4 = search_produits4(driver,df_rayons)
    df_produits = pd.concat([df_produits1, df_produits2,df_produits3, df_produits4], axis=0)
     
    
    

    

    


{'rayon principal': 'Promos', 'lien rayon principal': 'https://www.auchan.fr/boutique/promos', 'rayon secondaire': 'Promos', 'lien rayon secondaire': 'https://www.auchan.fr/boutique/promos'}
{'rayon principal': 'Nos solutions anti-inflation', 'lien rayon principal': 'https://www.auchan.fr/petits-prix-grandes-economies/ca-b17', 'rayon secondaire': 'Trimestre anti-inflation', 'lien rayon secondaire': 'https://www.auchan.fr/petits-prix-grandes-economies/trimestre-anti-inflation/ca-b202303101730'}
{'rayon principal': 'Nos solutions anti-inflation', 'lien rayon principal': 'https://www.auchan.fr/petits-prix-grandes-economies/ca-b17', 'rayon secondaire': 'Moins de 1€', 'lien rayon secondaire': 'https://www.auchan.fr/boutique/moins-de-1-euros'}
{'rayon principal': 'Nos solutions anti-inflation', 'lien rayon principal': 'https://www.auchan.fr/petits-prix-grandes-economies/ca-b17', 'rayon secondaire': 'Le frais à prix plaisir', 'lien rayon secondaire': 'https://www.auchan.fr/boutique/frais-a-pr

find index of a specific character in string 

In [11]:
def find_last_index(string, char, indices = []):
    for i in range(len(string)):
        if string[i] == char:
            indices.append(i)

    last_indice = int(indices[-1])
    return(last_indice)
    

Data Clean and Preprocessing

In [12]:
def data_prix_clean(df_produits, list_produits =[]):
    for (i,prix), prixrat in zip(enumerate(df_produits['prix_produit']),df_produits['attribut_produit']):
        #get prix
        try :
            prix_trans = float(prix[:-1].replace(",","."))
        except :
            prix_trans = None
        #get unite
        try :
            index_slash = find_last_index(string = prixrat, char='/')
            unite = prixrat[index_slash:]
        except : 
            unite = None

        
        prixrat = prixrat.replace("\n","%")
        
        #get prix ratio
        
        try :
            pourcent = find_last_index(string = prixrat, char= "%")
            prix_rat_trans = float(prixrat[(pourcent+1):(index_slash-2)].replace(",","."))
        except :
            prix_rat_trans = None


        #get quantité
        quantite = prixrat[:(pourcent-1)].replace("%","/")

        dict_produit={
                        'rayon_principal' : df_produits.iloc[i]['rayon_principal'],
                        'lien_rayon principal' : df_produits.iloc[i]['lien_rayon principal'],
                        'rayon_secondaire': df_produits.iloc[i]['rayon_secondaire'],
                        'lien_rayon_secondaire' : df_produits.iloc[i]['lien_rayon_secondaire'],
                        'nom_produit': df_produits.iloc[i]['nom_produit'],
                        'quantite' : quantite,
                        'prix_produit': prix_trans,
                        'prix_ratio' : prix_rat_trans,
                        'unite' : unite
                    }
        print(dict_produit)
        list_produits.append(dict_produit)
    df_produits = pd.DataFrame(list_produits)
    df_produits.to_csv('Auchan_tous_produits.csv')
    return(df_produits)
    

    


In [13]:
print(df_produits.columns)
data_prix_clean(df_produits)

Index(['rayon_principal', 'lien_rayon principal', 'rayon_secondaire',
       'lien_rayon_secondaire', 'nom_produit', 'prix_produit',
       'attribut_produit'],
      dtype='object')
{'rayon_principal': 'Promos', 'lien_rayon principal': 'https://www.auchan.fr/boutique/promos', 'rayon_secondaire': 'Promos', 'lien_rayon_secondaire': 'https://www.auchan.fr/boutique/promos', 'nom_produit': "JOKER Pur jus d'orange sans pulpe", 'quantite': '4x1.5', 'prix_produit': 13.16, 'prix_ratio': 2.19, 'unite': '/ l'}
{'rayon_principal': 'Promos', 'lien_rayon principal': 'https://www.auchan.fr/boutique/promos', 'rayon_secondaire': 'Promos', 'lien_rayon_secondaire': 'https://www.auchan.fr/boutique/promos', 'nom_produit': 'MONTAUDON AOP Champagne brut', 'quantite': '75c', 'prix_produit': 18.4, 'prix_ratio': 24.53, 'unite': '/ l'}
{'rayon_principal': 'Promos', 'lien_rayon principal': 'https://www.auchan.fr/boutique/promos', 'rayon_secondaire': 'Promos', 'lien_rayon_secondaire': 'https://www.auchan.fr/bouti

ValueError: could not convert string to float: ''